In [2]:
import torch #for using tensors
import torch.nn as nn # creating neural network
import torch.nn.functional as F # recallable functions like relu,sigmoid and etc.
import torchvision # subpackage for vision models
import torchvision.transforms as transforms # image augmentation
import torch.optim as optim # optimisation functions like sgd, adam
from torch.utils.data import DataLoader # creating a data loader
from torchvision.transforms import ToTensor # converting image to tensor
import matplotlib.pyplot as plt 

In [13]:
# Download training data from open datasets.
training_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor(),
)

In [14]:
test_data[0][0].shape
# test data is already formed as a tensor dataset which means it has 2 dimensions 
# first dimension is X variable ( images pixel values)
# second one is it's label in which it is going to learn 
# when we write test_data[0] we are calling the first images X and y values
# when we write test_data[0][0] we are calling first images X only and test_data[0][1] we are calling it's y
# X variable has got 3 dimension 
# 1'st one is collor chanel
# 2'nd and 3rd ones are height and width pixel counts
# in this case we have gray scale image with 28x28 pixel dimension

torch.Size([1, 28, 28])

In [15]:
# Hypers
batch_size = 64

In [16]:
train_loader = DataLoader(training_data,shuffle=True,batch_size=batch_size)
test_loader = DataLoader(test_data,shuffle=True)
# data loader is a tool which defines how our data will get into the model
# in training fase we should define batch_size for increasing the model's speed and reducing underfiting 
# but in testing fase it is not necessary as we will not do backprop

In [17]:
#     0: "T-Shirt",
#     1: "Trouser",
#     2: "Pullover",
#     3: "Dress",
#     4: "Coat",
#     5: "Sandal",
#     6: "Shirt",
#     7: "Sneaker",
#     8: "Bag",
#     9: "Ankle Boot",

In [18]:
for x,y in train_loader:
    print(x.shape)
    print(y)
    break

torch.Size([64, 1, 28, 28])
tensor([0, 9, 8, 3, 9, 5, 4, 6, 3, 3, 4, 6, 9, 0, 1, 7, 6, 4, 2, 5, 8, 5, 0, 3,
        9, 7, 3, 9, 3, 2, 0, 2, 6, 5, 4, 3, 9, 1, 5, 9, 2, 3, 5, 7, 6, 4, 3, 0,
        2, 2, 0, 5, 9, 8, 4, 2, 9, 4, 7, 8, 0, 9, 1, 7])


# Modelling

In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()# first 3 lines is mandatory for inheriting properties of model
# conv layer are special layer's which uses filter for collectiong various information from images
# in_channels on first layer should be number of colur channels in our case it is 1
# kernel size is a size of filter matrix 5x5 in this case
# padding adds a pixel in every direction in oorder learning corner's better
# stride is defining movement of kernel. default is 1
# image size after each conv layer is detected by s = (Si-filter_size+2*padding)/stride + 1
# and after pooling if we use 2,2 it will shrink from every side 2 times
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(64,128,kernel_size=5,stride=2,padding=2)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(128*7*7,512)
        self.fc2 = nn.Linear(512,512)
        self.out = nn.Linear(512,10)
        self.drop_out = nn.Dropout()
    def forward(self,X):
        # input image size is [1,28,28]
        X = self.conv1(X)
        # after first layer of conv z absis will get number from out_channels (64)
        # (28-5+2*2)/1+1 -> 28
        # our result will be [64,28,28]
        X = self.conv2(X)
        # out_channel is 128
        # (28-5+2*2)/2 + 1 ->14.5 aka 14
        # [128,14,14]
        X = self.pool(X)
        # 14/2 -> 7
        # [128,7,7]
        X = X.reshape(X.size(0), -1)
        # [ 128*7*7] -> one dimension
        X = self.drop_out(X)
        # Regularization and preventing the co-adaptation of neurons as described in the paper
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        out = self.out(X)
        return X

In [20]:
(28-5+2*2)/2+1

14.5

In [21]:
model = NeuralNetwork()
# defining model

In [22]:
num_epochs = 6  # how many times model will go back and forth
num_classes = 10 # number of labels we should predict
learning_rate = 0.001 #step of learning

In [23]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() #loss function for defining if we  predicted right or wronge
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # optimisation function for creating gradient decent and learning

In [24]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/6], Step [100/938], Loss: 0.9275, Accuracy: 73.44%
Epoch [1/6], Step [200/938], Loss: 0.3183, Accuracy: 89.06%
Epoch [1/6], Step [300/938], Loss: 0.2421, Accuracy: 92.19%
Epoch [1/6], Step [400/938], Loss: 0.3175, Accuracy: 85.94%
Epoch [1/6], Step [500/938], Loss: 0.3206, Accuracy: 89.06%
Epoch [1/6], Step [600/938], Loss: 0.4381, Accuracy: 85.94%
Epoch [1/6], Step [700/938], Loss: 0.3380, Accuracy: 84.38%
Epoch [1/6], Step [800/938], Loss: 0.4396, Accuracy: 82.81%
Epoch [1/6], Step [900/938], Loss: 0.4532, Accuracy: 90.62%
Epoch [2/6], Step [100/938], Loss: 0.5183, Accuracy: 78.12%
Epoch [2/6], Step [200/938], Loss: 0.2295, Accuracy: 90.62%
Epoch [2/6], Step [300/938], Loss: 0.3286, Accuracy: 89.06%
Epoch [2/6], Step [400/938], Loss: 0.3901, Accuracy: 87.50%
Epoch [2/6], Step [500/938], Loss: 0.2334, Accuracy: 92.19%
Epoch [2/6], Step [600/938], Loss: 0.2473, Accuracy: 90.62%
Epoch [2/6], Step [700/938], Loss: 0.3737, Accuracy: 84.38%
Epoch [2/6], Step [800/938], Loss: 0.377

In [25]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))


Test Accuracy of the model on the 10000 test images: 90.86999999999999 %
